# '''dahyun+darwin= dahwin'''

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips,vfx , AudioFileClip
from moviepy import editor
from moviepy.editor import transfx
x = dir(transfx)
clip1 = VideoFileClip('fancy.mp4').subclip(0,6).fx(vfx.fadein,1).fx(vfx.fadeout,1)
clip2 = VideoFileClip('feel.mp4').subclip(0,6).fx(vfx.fadein,1).fx(vfx.fadeout,1)

combined = concatenate_videoclips([clip1,clip2])
combined.write_videofile('dahyun.mp4')

In [6]:
from moviepy.editor import ImageClip, concatenate_videoclips

# Step 1: Set the image filenames
image_filenames = ["nn.jpg", "n.jpeg"]

# Step 2: Set the duration for each image (in seconds)
image_durations = [5, 5]  # Adjust the durations as needed

# Step 3: Create ImageClip objects from the images
image_clips = []
for filename, duration in zip(image_filenames, image_durations):
    image = ImageClip(filename, duration=duration)
    image_clips.append(image)

# Step 4: Concatenate the image clips into a single video clip
video_clip = concatenate_videoclips(image_clips, method="chain")

# Step 5: Set the output video filename
output_filename = "test.mp4"
# Set the target resolution to HD (1920x1080)
target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
combined = video_clip.resize(height=target_resolution[1], width=target_resolution[0])
# Step 6: Write the video file
combined.write_videofile(output_filename, fps=10, codec="libx264",
                           audio_codec="aac", bitrate="5000k")



Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


# Side in

In [7]:
from moviepy.editor import VideoFileClip, vfx

import cv2
import numpy as np

clip1 = VideoFileClip('likey.mp4').subclip(0, 6)
clip2 = VideoFileClip('fancy.mp4').subclip(0, 6)

duration = 30  # in frames

frames1 = [frame for frame in clip1.iter_frames()][:-duration]
duration_frames = [frame for frame in clip1.iter_frames()][-duration:]

frames2 = [frame for frame in clip2.iter_frames()]

height, width, _ = frames1[0].shape
transition_frames = []

for i in range(duration):
    # Calculate the translation distance based on the frame index
    # For slide-in effect from right, clip1 moves to the left and clip2 comes from the right
    tx = int((width / duration) * i)
    tx_clip1 = -tx
    tx_clip2 = width - tx

    # Create a blank frame
    blank_frame = np.zeros_like(frames1[0])

    # # Convert duration frame from RGB to BGR
    # frame_clip1 = cv2.cvtColor(duration_frames[i], cv2.COLOR_RGB2BGR)

    # Apply translation to clip1
    frame_clip1 = cv2.warpAffine(duration_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))

    # Apply translation to clip2
    frame_clip2 = cv2.warpAffine(frames2[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))

    alpha = i / duration  # Linearly increasing alpha value

    blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)

    transition_frames.append(blend_frame)

remaining_frames = frames2[duration:]

final_frames = frames1 + transition_frames + remaining_frames
fps = clip1.fps

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

for frame in final_frames:
    # Convert frame from RGB to BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    output.write(frame)

output.release()



In [ ]:
from moviepy.editor import *
import cv2
import numpy as np

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "motion.mp4"]

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    video_clips.append(video)

# Step 3: Define transition duration
transition_duration = 2  # Duration of the slide-in transition in seconds

# Step 4: Extract the last one second of clip1 and the first one second of clip2
clip1_last_second = video_clips[0].subclip(-2, None)
clip2_first_second = video_clips[1].subclip(0, 2)

# Step 5: Convert the clips to numpy arrays
clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
clip2_frames = [frame for frame in clip2_first_second.iter_frames()]

# Step 6: Perform slide-in transition using numpy and cv2
transition_frames = []
height, width, _ = clip1_frames[0].shape

# Ensure the number of frames matches the expected duration
num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))

for i in range(num_frames):
    alpha = i / (transition_duration * clip1_last_second.fps)  # Linearly increasing alpha value

    # Calculate the translation distance based on the frame index
    tx = int((width / (transition_duration * clip1_last_second.fps)) * i)
    tx_clip1 = -tx
    tx_clip2 = width - tx

    # Apply translation to clip1
    frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))

    # Handle the case where clip2_frames has fewer frames than expected
    if i < len(clip2_frames):
        # Apply translation to clip2
        frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    else:
        # Use the last frame of clip2_frames for the remaining frames
        frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))

    # Blend the frames to create the transition effect
    blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)

    transition_frames.append(blend_frame)

# Step 7: Concatenate the original clips and the transition frames to create the final video
final_clip = concatenate_videoclips([video_clips[0], VideoClip(lambda t: transition_frames[int(t * clip1_last_second.fps)], duration=transition_duration), video_clips[1]])

# Step 8: Write the final video to a file
final_clip.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)




t:  41%|███████████████████████████▏                                       | 255/629 [00:21<00:13, 28.53it/s, now=None]

In [1]:
from moviepy.editor import *
import cv2
import numpy as np

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "motion.mp4"]

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    video_clips.append(video)

# Step 3: Define transition duration
transition_duration = 1  # Duration of the slide-in transition in seconds

# Step 4: Extract the last one second of clip1 and the first one second of clip2
clip1_last_second = video_clips[0].subclip(-1, None)
clip2_first_second = video_clips[1].subclip(0, 1)

# Step 5: Convert the clips to numpy arrays
clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
clip2_frames = [frame for frame in clip2_first_second.iter_frames()]

# Step 6: Perform slide-in transition using numpy and cv2
transition_frames = []
height, width, _ = clip1_frames[0].shape

# Ensure the number of frames matches the expected duration
num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))

for i in range(num_frames):
    alpha = i / num_frames  # Linearly increasing alpha value

    # Calculate the translation distance based on the frame index
    tx = int((width / num_frames) * i)
    tx_clip1 = -tx
    tx_clip2 = width - tx

    # Apply translation to clip1
    frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))

    # Handle the case where clip2_frames has fewer frames than expected
    if i < len(clip2_frames):
        # Apply translation to clip2
        frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    else:
        # Use the last frame of clip2_frames for the remaining frames
        frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))

    # Blend the frames to create the transition effect
    blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)

    transition_frames.append(blend_frame)

# Step 7: Concatenate the original clips and the transition frames to create the final video
transition_clip = ImageSequenceClip(transition_frames, fps=clip1_last_second.fps)
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), transition_clip.set_duration(transition_duration), video_clips[1].subclip(1, None)],method='compose')

# Step 8: Write the final video to a file
final_clip.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)



Moviepy - Building video lastslidein.mp4.
MoviePy - Writing audio in lastslideinTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video lastslidein.mp4



Moviepy - Done !
Moviepy - video ready lastslidein.mp4


In [30]:
import re

def convert_string_to_final_clip(input_string):
    # Define regular expression patterns to match video_clips[i] and custom function expressions
    video_clip_pattern = r'video_clips\[(\d+)\]'
    custom_function_pattern = r'(\w+)\(\)'

    # Find all occurrences of video_clips[i] and custom function expressions in the input string
    video_clip_matches = re.findall(video_clip_pattern, input_string)
    custom_function_matches = re.findall(custom_function_pattern, input_string)

    # Check if the input string contains at least one video_clips[i] and one custom function expression
    if not video_clip_matches or not custom_function_matches:
        return "Invalid input string. It should contain at least one video_clips[i] and one custom function expression."

    # Construct the final_clip string
    final_clip_str = "concatenate_videoclips(["

    num_clips = min(len(video_clip_matches), len(custom_function_matches))
    for i in range(num_clips):
        video_idx = int(video_clip_matches[i])
        custom_function_name = custom_function_matches[i]

        custom_function_str = "{}(video_clips[{}], video_clips[{}])".format(custom_function_name, video_idx, video_idx + 1)

        subclip_str1 = "video_clips[{}].subclip(0, video_clips[{}].duration - 1)".format(video_idx, video_idx)

        if i == num_clips - 1:  # Last video clip
            subclip_str2 = "video_clips[{}].subclip(1.3, None)".format(video_idx + 1)
        else:
            subclip_str2 = "video_clips[{}].subclip(1.3, video_clips[{}].duration - 1)".format(video_idx + 1, video_idx + 1)

        final_clip_str += "{}, {}, {}, ".format(subclip_str1, custom_function_str, subclip_str2)

    # Remove the last two characters ", " from the final_clip_str
    final_clip_str = final_clip_str[:-2]

    # Add the method argument and closing bracket
    final_clip_str += "], method='compose')"

    return final_clip_str

# Test examples
input_string1 = 'video_clips[0]+slidein()+video_clips[1]+slidein()+video_clips[2]'
input_string2 = 'video_clips[0]+slidein()+video_clips[1]+fadedown()+video_clips[2]+fadein()+video_clips[3]+fun()+video_clips[4]+funck()+video_clips[5]'

print(convert_string_to_final_clip(input_string1))
print(convert_string_to_final_clip(input_string2))
# concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1),slidein(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')

concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), video_clips[1].subclip(0, video_clips[1].duration - 1), slidein(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')
concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), video_clips[1].subclip(0, video_clips[1].duration - 1), fadedown(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, video_clips[2].duration - 1), video_clips[2].subclip(0, video_clips[2].duration - 1), fadein(video_clips[2], video_clips[3]), video_clips[3].subclip(1.3, video_clips[3].duration - 1), video_clips[3].subclip(0, video_clips[3].duration - 1), fun(video_clips[3], video_clips[4]), video_clips[4].subclip(1.3, video_clips[4].duration - 1), video_clips[4].subclip(0, vide

In [108]:
import re

def convert_string_to_final_clip(input_string):
    # Define regular expression patterns to match video_clips[i] and custom function expressions
    video_clip_pattern = r'video_clips\[(\d+)\]'
    custom_function_pattern = r'(\w+)\(\)'

    # Find all occurrences of video_clips[i] and custom function expressions in the input string
    video_clip_matches = re.findall(video_clip_pattern, input_string)
    custom_function_matches = re.findall(custom_function_pattern, input_string)

    # Check if the input string contains at least one video_clips[i] and one custom function expression
    if not video_clip_matches or not custom_function_matches:
        return "Invalid input string. It should contain at least one video_clips[i] and one custom function expression."

    # Construct the final_clip string
    final_clip_str = "concatenate_videoclips(["

    num_clips = len(video_clip_matches)

    for i, (video_idx, custom_function_name) in enumerate(zip(video_clip_matches, custom_function_matches)):
        video_idx = int(video_idx)

        custom_function_str = "{}(video_clips[{}], video_clips[{}])".format(custom_function_name, video_idx, video_idx + 1)

        if i == 0:  # First video clip
            subclip_str1 = "video_clips[{}].subclip(0, video_clips[{}].duration - 1)".format(video_idx, video_idx)
        else:
            subclip_str1 = "video_clips[{}].subclip(1.3, video_clips[{}].duration - 1)".format(video_idx, video_idx)

        if i == num_clips - 1:  # Last video clip
            subclip_str2 = "video_clips[{}].subclip(1.3, None)".format(video_idx)
        else:
            next_video_idx = int(video_clip_matches[i + 1])
            subclip_str2 = "video_clips[{}].subclip(1.3, video_clips[{}].duration - 1)".format(video_idx + 1, next_video_idx)

        final_clip_str += "{}, {}, ".format(subclip_str1, custom_function_str)
    
    # Add the last video clip
    final_clip_str += "{}], method='compose')".format(subclip_str2)

    return final_clip_str

# Test example
input_string1 = 'video_clips[0]+slidein()+video_clips[1]+slidein()+video_clips[2]+slidein()+video_clips[3]'
string = convert_string_to_final_clip(input_string1)
print(string)




concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), slidein(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, video_clips[2].duration - 1), slidein(video_clips[2], video_clips[3]), video_clips[3].subclip(1.3, video_clips[3].duration - 1)], method='compose')


In [114]:
import re


video_clips_string = string
# Use regular expression to extract the last part of the list
pattern = r"video_clips\[\d+\]\.subclip\([^)]+\)"
matches = re.findall(pattern, video_clips_string)

if matches:
    last_part_of_list = matches[-1]
    print(last_part_of_list)
else:
    print("No match found.")

def get_number_from_string(string):

  match = re.search(r"\d+", string)
  if match:
    return int(match.group(0))
  else:
    return None

number = get_number_from_string(last_part_of_list)

print(number)

import re

input_string = video_clips_string

# Define the regular expression pattern to match the desired part
pattern = f"video_clips\[{number}\].subclip\(1\.3, video_clips\[{number}\].duration - 1\)"

# Replace the matched part with the new desired part
clips = re.sub(pattern, f"video_clips[{number}].subclip(1.3, None)", input_string)

print(clips)

video_clips[3].subclip(1.3, video_clips[3].duration - 1)
3
concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), slidein(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, video_clips[2].duration - 1), slidein(video_clips[2], video_clips[3]), video_clips[3].subclip(1.3, None)], method='compose')


# center

# Function

In [112]:
import cv2
import numpy as np
from moviepy.editor import *

# Step 1: Set the video filenames
video_filenames = ['fancy.mp4','resized_tofu.mp4','fancy.mp4','feel.mp4']

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    
    video_clips.append(video)

def slidein(video0,video1):
        # Step 3: Define transition duration
    transition_duration = 1  # Duration of the slide-in transition in seconds
    
    # Step 4: Extract the last one second of clip1 and the first one second of clip2
    clip1_last_second = video0.subclip(-1, None)
    clip2_first_second = video1.subclip(0, 1.3)
    
    # Check if aspect ratio of clip1_last_second is less than 1.78:1 and add black padding
    if clip1_last_second.aspect_ratio < 1.78:
        width = int(1080 * clip1_last_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip1_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip1_last_second.iter_frames()]
        # clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    else:
        clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    
    # Check if aspect ratio of clip2_first_second is less than 1.78:1 and add black padding
    if clip2_first_second.aspect_ratio < 1.78:
        width = int(1080 * clip2_first_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip2_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip2_first_second.iter_frames()]
        # clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    else:
        
        clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    
    # Step 6: Perform slide-in transition using numpy and cv2
    transition_frames = []
    height, width, _ = clip1_frames[0].shape
    
    # Ensure the number of frames matches the expected duration
    num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))
    
    for i in range(num_frames):
        alpha = i / num_frames  # Linearly increasing alpha value
    
        # Calculate the translation distance based on the frame index
        tx_clip1 = -int((width / num_frames) * i)
        tx_clip2 = width - int((width / num_frames) * i)
    
        # Apply translation to clip1
        frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))
    
        # Handle the case where clip2_frames has fewer frames than expected
        if i < len(clip2_frames):
            # Apply translation to clip2
            frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
        else:
            # Use the last frame of clip2_frames for the remaining frames
            frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    
        # Blend the frames to create the transition effect
        blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)
    
        transition_frames.append(blend_frame)
    
    # Add remaining frames where clip2_first_second fills the entire screen
    remaining_frames = clip2_frames[num_frames:]
    transition_frames.extend(remaining_frames)
    
    
    # Step 7: Concatenate the original clips and the transition frames to create the final video
    transition_clip = ImageSequenceClip(transition_frames, fps=clip1_last_second.fps)
    transition_clip = transition_clip.set_duration(transition_duration)
    return transition_clip
    
# Use the convert_string_to_final_clip function to get the final_clip
# final_clip_str = convert_string_to_final_clip(input_string1)

# Evaluate the final_clip_str to get the final_clip video clip object
final_clip = eval(clips)


target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
final_clips = final_clip.resize(height=target_resolution[1], width=target_resolution[0])
# Step 8: Write the final video to a file
final_clips.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)




Moviepy - Building video lastslidein.mp4.
MoviePy - Writing audio in lastslideinTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video lastslidein.mp4



Moviepy - Done !
Moviepy - video ready lastslidein.mp4


In [6]:
# final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, None)], method='compose')
import cv2
import numpy as np
from moviepy.editor import *

# Step 1: Set the video filenames
video_filenames = ['people-in-a-conference-room-for-a-business-meeting_duration_21_second.mp4','a-monk-meditating-on-a-tree_duration_15_second.mp4','sweet-foods-causing-diabetes_duration_9_second.mp4','a-monk-meditating-on-a-tree_duration_15_second.mp4']

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    
    video_clips.append(video)

def slidein(video0,video1):
        # Step 3: Define transition duration
    transition_duration = 1  # Duration of the slide-in transition in seconds
    
    # Step 4: Extract the last one second of clip1 and the first one second of clip2
    clip1_last_second = video0.subclip(-1, None)
    clip2_first_second = video1.subclip(0, 1.3)
    
    # Check if aspect ratio of clip1_last_second is less than 1.78:1 and add black padding
    if clip1_last_second.aspect_ratio < 1.78:
        width = int(1080 * clip1_last_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip1_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip1_last_second.iter_frames()]
        # clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    else:
        clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    
    # Check if aspect ratio of clip2_first_second is less than 1.78:1 and add black padding
    if clip2_first_second.aspect_ratio < 1.78:
        width = int(1080 * clip2_first_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip2_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip2_first_second.iter_frames()]
        # clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    else:
        
        clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    
    # Step 6: Perform slide-in transition using numpy and cv2
    transition_frames = []
    height, width, _ = clip1_frames[0].shape
    
    # Ensure the number of frames matches the expected duration
    num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))
    
    for i in range(num_frames):
        alpha = i / num_frames  # Linearly increasing alpha value
    
        # Calculate the translation distance based on the frame index
        tx_clip1 = -int((width / num_frames) * i)
        tx_clip2 = width - int((width / num_frames) * i)
    
        # Apply translation to clip1
        frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))
    
        # Handle the case where clip2_frames has fewer frames than expected
        if i < len(clip2_frames):
            # Apply translation to clip2
            frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
        else:
            # Use the last frame of clip2_frames for the remaining frames
            frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    
        # Blend the frames to create the transition effect
        blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)
    
        transition_frames.append(blend_frame)
    
    # Add remaining frames where clip2_first_second fills the entire screen
    remaining_frames = clip2_frames[num_frames:]
    transition_frames.extend(remaining_frames)
    
    
    # Step 7: Concatenate the original clips and the transition frames to create the final video
    transition_clip = ImageSequenceClip(transition_frames, fps=clip1_last_second.fps)
    transition_clip = transition_clip.set_duration(transition_duration)
    return transition_clip
    
# Use the convert_string_to_final_clip function to get the final_clip
# final_clip_str = convert_string_to_final_clip(input_string1)

# Evaluate the final_clip_str to get the final_clip video clip object
final_clip = final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1),slidein(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')


target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
final_clips = final_clip.resize(height=target_resolution[1], width=target_resolution[0])
# Step 8: Write the final video to a file
final_clips.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)

Exception: Moviepy: ImageSequenceClip requires all images to be the same size

In [14]:
import cv2
import numpy as np
from moviepy.editor import *

# Step 1: Set the video filenames
video_filenames = ['black-and-white-video-of-busy-street_duration_14_second.mp4','a-monk-meditating-on-a-tree_duration_15_second.mp4','sweet-foods-causing-diabetes_duration_9_second.mp4','a-monk-meditating-on-a-tree_duration_15_second.mp4']

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    
    video_clips.append(video)

def slidein(video0,video1):
        # Step 3: Define transition duration
    transition_duration = 1  # Duration of the slide-in transition in seconds
    
    # Step 4: Extract the last one second of clip1 and the first one second of clip2
    clip1_last_second = video0.subclip(-1, None)
    clip2_first_second = video1.subclip(0, 1.3)
    
    # Check if aspect ratio of clip1_last_second is less than 1.78:1 and add black padding
    if clip1_last_second.aspect_ratio < 1.78:
        width = int(1080 * clip1_last_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip1_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip1_last_second.iter_frames()]
        # clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    else:
        clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
    
    # Check if aspect ratio of clip2_first_second is less than 1.78:1 and add black padding
    if clip2_first_second.aspect_ratio < 1.78:
        width = int(1080 * clip2_first_second.aspect_ratio)
        padding = (1920 - width) // 2
        clip2_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip2_first_second.iter_frames()]
        # clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    else:
        
        clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
    
    # Step 6: Perform slide-in transition using numpy and cv2
    transition_frames = []
    height, width, _ = clip1_frames[0].shape
    
 # Ensure the number of frames matches the expected duration
    num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))
    
    for i in range(num_frames):
        alpha = i / num_frames  # Linearly increasing alpha value
    
        # Calculate the translation distance based on the frame index
        tx_clip1 = -int((width / num_frames) * i)
        tx_clip2 = width - int((width / num_frames) * i)
    
        # Apply translation to clip1
        frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))
    
        # Handle the case where clip2_frames has fewer frames than expected
        if i < len(clip2_frames):
            # Apply translation to clip2
            frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
        else:
            # Use the last frame of clip2_frames for the remaining frames
            frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    
        # Blend the frames to create the transition effect
        blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)
    
        transition_frames.append(blend_frame)
    
    # Add remaining frames where clip2_first_second fills the entire screen
    remaining_frames = clip2_frames[num_frames:]
    transition_frames.extend(remaining_frames)
    
     # Step 7: Create the transition clip using NumPy
    transition_clip = VideoClip(make_frame=lambda t: transition_frames[int(t * clip1_last_second.fps)], duration=transition_duration)

    return transition_clip

    
# Use the convert_string_to_final_clip function to get the final_clip
# final_clip_str = convert_string_to_final_clip(input_string1)

# Evaluate the final_clip_str to get the final_clip video clip object
final_clip = final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1),slidein(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')


target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
final_clips = final_clip.resize(height=target_resolution[1], width=target_resolution[0])
# Step 8: Write the final video to a file
final_clips.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)

Moviepy - Building video lastslidein.mp4.
Moviepy - Writing video lastslidein.mp4



Moviepy - Done !
Moviepy - video ready lastslidein.mp4


In [23]:
from moviepy.editor import *

# Step 1: Set the video filename
video_filename = "tofu.mp4"

# Step 2: Create a VideoFileClip object from the video
video_clip = VideoFileClip(video_filename)


# Step 4: Resize the video clip while maintaining the aspect ratio
resized_clip = video_clip.resize(width=1080, height=1920)

# Step 5: Write the resized video to a new file
resized_clip.write_videofile("resized_tofu.mp4", codec="libx264", fps=resized_clip.fps)


Moviepy - Building video resized_tofu.mp4.
MoviePy - Writing audio in resized_tofuTEMP_MPY_wvf_snd.mp3


chunk:  56%|██████████████████████████████████▌                           | 236/423 [00:00<00:00, 855.56it/s, now=None]
KeyboardInterrupt

chunk:  60%|█████████████████████████████████████▏                        | 254/423 [00:17<00:00, 855.56it/s, now=None]

In [118]:
import cv2

# Step 1: Set the video filename
video_filename = "tofu.mp4"

# Step 2: Create a VideoCapture object from the video
cap = cv2.VideoCapture(video_filename)

# Step 3: Get the height and width of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Step 4: Calculate the new width and height with the desired aspect ratio
# Step 4: Calculate the new width and height with the desired aspect ratio
desired_height = 1080
aspect_ratio = width / height
new_width = int(desired_height * aspect_ratio)

# Step 5: Create VideoWriter object to write the resized video
output_filename = "resized_tofu.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, desired_height))

# Step 6: Read frames from the original video, resize, and write to the new video
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    resized_frame = cv2.resize(frame, (new_width, desired_height))
    out.write(resized_frame)

# Step 7: Release VideoCapture and VideoWriter objects
cap.release()
out.release()

# Print the height and width of the resized video
print("Height:", desired_height)
print("Width:", new_width)


Height: 1080
Width: 607


In [117]:
import cv2

# Step 1: Set the video filename
video_filename = "tofu.mp4"

# Step 2: Create a VideoCapture object from the video
cap = cv2.VideoCapture(video_filename)

# Step 3: Get the height and width of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Step 4: Calculate the new width and height with the desired aspect ratio
desired_height = 1080
aspect_ratio = width / height
new_width = int(desired_height * aspect_ratio)

# Step 5: Create VideoWriter object to write the resized video
output_filename = "resized_tofu.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, desired_height))

# Step 6: Read frames from the original video, resize, and write to the new video
resized_frame = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    resized_frame = cv2.resize(frame, (new_width, desired_height))
    resized_frame.append(resized_frame)

video = ImageSequenceClip(resized_frame,fps=cap.fps)
# Step 7: Release VideoCapture and VideoWriter objects
cap.release()
out.release()

# Print the height and width of the resized video
print("Height:", desired_height)
print("Width:", new_width)


AttributeError: 'numpy.ndarray' object has no attribute 'append'

# Moviepy

In [5]:
from moviepy.editor import ImageSequenceClip
import cv2

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "motion.mp4", "feel.mp4"]

# Step 2: Create a list to store the ImageSequenceClip objects for each video
video_clips = []

for filename in video_filenames:
    # Step 3: Create a VideoCapture object from the video
    cap = cv2.VideoCapture(filename)

    # Step 4: Get the height and width of the video
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Step 5: Calculate the new width and height with the desired aspect ratio
    desired_height = 1080
    aspect_ratio = width / height
    new_width = int(desired_height * aspect_ratio)

    # Step 6: Read frames from the original video, resize, and store in a list
    resized_frames = []
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (new_width, desired_height))

        # Append the resized_frame to the list
        resized_frames.append(resized_frame)

    # Step 7: Release VideoCapture object
    cap.release()

    # Step 8: Create an ImageSequenceClip from the list of resized frames and add it to the video_clips list
    video_clips.append(ImageSequenceClip(resized_frames, fps=fps))

# Print the height and width of the resized videos
print("Height:", desired_height)
print("Width:", new_width)

# Step 9: Now you have a list of ImageSequenceClip objects for each video
# You can use these video_clips for further processing or editing with moviepy
# For example, you can concatenate them to create a single video or apply other transformations.

# For example, to concatenate the clips and write the final video:
final_clip = concatenate_videoclips(video_clips, method="compose")
final_clip.write_videofile("concatenated_video.mp4", codec="libx264", fps=final_clip.fps)



Height: 1080
Width: 1920
Moviepy - Building video concatenated_video.mp4.
Moviepy - Writing video concatenated_video.mp4



Moviepy - Done !
Moviepy - video ready concatenated_video.mp4


In [4]:
import cv2

# Load the video
video_path = "sweet-foods-causing-diabetes_duration_9_second.mp4"
cap = cv2.VideoCapture(video_path)

# Get the width and height of the video
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("Height:", height)
print("Width:", width)
# Calculate the aspect ratio
aspect_ratio = width / height

print("Aspect Ratio: {:.2f}:{:.2f}".format(aspect_ratio, 1))

# Release the video capture
cap.release()


Height: 1080.0
Width: 607.0
Aspect Ratio: 0.56:1.00


In [1]:
string = 'video_clips[0]+slidein()+video_clips[1]'
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, None)], method='compose')

string = 'video_clips[1]+slidein()+video_clips[2]'
final_clip = concatenate_videoclips([video_clips[1].subclip(0, video_clips[1].duration - 1), slidein(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')

string = 'video_clips[0]+slidein()+video_clips[1]+fadedown()+video_clips[2]'
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1),fadedown(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')

string = 'video_clips[0]+slidein()+video_clips[1]+fadedown()+video_clips[2]+fadein()+video_clips[3]'
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0],video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1),fadedown(video_clips[1],video_clips[2]), video_clips[2].subclip(1.3, video_clips[2].duration - 1),fadein(video_clips[2],video_clips[3]),video_clips[3].subclip(1.3, None))], method='compose')


In [12]:
import re

def convert_string_to_final_clip(input_string):
    # Define regular expression patterns to match video_clips[i] and custom function expressions
    video_clip_pattern = r'video_clips\[(\d+)\]'
    custom_function_pattern = r'(\w+)\(\)'

    # Find all occurrences of video_clips[i] and custom function expressions in the input string
    video_clip_matches = re.findall(video_clip_pattern, input_string)
    custom_function_matches = re.findall(custom_function_pattern, input_string)

    # Check if the input string contains at least one video_clips[i] and one custom function expression
    if not video_clip_matches or not custom_function_matches:
        return "Invalid input string. It should contain at least one video_clips[i] and one custom function expression."

    # Construct the final_clip string
    final_clip_str = "concatenate_videoclips(["

    num_clips = min(len(video_clip_matches), len(custom_function_matches))
    for i in range(num_clips):
        video_idx = int(video_clip_matches[i])
        custom_function_name = custom_function_matches[i]

        custom_function_str = "{}(video_clips[{}], video_clips[{}]), ".format(custom_function_name, video_idx, video_idx + 1)

        subclip_str1 = "video_clips[{}].subclip(0, video_clips[{}].duration - 1), ".format(video_idx, video_idx)
        subclip_str2 = "video_clips[{}].subclip(1.3, None), ".format(video_idx + 1)

        final_clip_str += "{}{}{}, ".format(subclip_str1, custom_function_str, subclip_str2)

    # Remove the last two characters ", " from the final_clip_str
    final_clip_str = final_clip_str[:-2]

    # Add the method argument and closing bracket
    final_clip_str += "], method='compose')"

    return final_clip_str

# Test examples
input_string1 = 'video_clips[0]+slidein()+video_clips[1]'
input_string2 = 'video_clips[1]+slidein()+video_clips[2]'

print(convert_string_to_final_clip(input_string1))
print(convert_string_to_final_clip(input_string2))



concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), fuckdein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, None), ], method='compose')
concatenate_videoclips([video_clips[1].subclip(0, video_clips[1].duration - 1), slidein(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, None), ], method='compose')


In [16]:
import re

def convert_string_to_final_clip(input_string):
    # Define regular expression patterns to match video_clips[i] and custom function expressions
    video_clip_pattern = r'video_clips\[(\d+)\]'
    custom_function_pattern = r'(\w+)\(\)'

    # Find all occurrences of video_clips[i] and custom function expressions in the input string
    video_clip_matches = re.findall(video_clip_pattern, input_string)
    custom_function_matches = re.findall(custom_function_pattern, input_string)

    # Check if the input string contains at least one video_clips[i] and one custom function expression
    if not video_clip_matches or not custom_function_matches:
        return "Invalid input string. It should contain at least one video_clips[i] and one custom function expression."

    # Construct the final_clip string
    final_clip_str = "concatenate_videoclips(["

    num_clips = min(len(video_clip_matches), len(custom_function_matches))
    for i in range(num_clips):
        video_idx = int(video_clip_matches[i])
        custom_function_name = custom_function_matches[i]

        custom_function_str = "{}(video_clips[{}], video_clips[{}])".format(custom_function_name, video_idx, video_idx + 1)

        subclip_str1 = "video_clips[{}].subclip(0, video_clips[{}].duration - 1)".format(video_idx, video_idx)

        if i == num_clips - 1:  # Last video clip
            subclip_str2 = "video_clips[{}].subclip(1.3, None)".format(video_idx + 1)
        else:
            subclip_str2 = "video_clips[{}].subclip(1.3, video_clips[{}].duration - 1)".format(video_idx + 1, video_idx + 1)

        final_clip_str += "{}, {}, {}, ".format(subclip_str1, custom_function_str, subclip_str2)

    # Remove the last two characters ", " from the final_clip_str
    final_clip_str = final_clip_str[:-2]

    # Add the method argument and closing bracket
    final_clip_str += "], method='compose')"

    return final_clip_str

# Test examples
input_string1 = 'video_clips[0]+slidein()+video_clips[1]+slidein()+video_clips[2]'
input_string2 = 'video_clips[0]+slidein()+video_clips[1]+fadedown()+video_clips[2]+fadein()+video_clips[3]+fun()+video_clips[4]+funck()+video_clips[5]'

print(convert_string_to_final_clip(input_string1))
print(convert_string_to_final_clip(input_string2))


concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), video_clips[1].subclip(0, video_clips[1].duration - 1), slidein(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, None)], method='compose')
concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), slidein(video_clips[0], video_clips[1]), video_clips[1].subclip(1.3, video_clips[1].duration - 1), video_clips[1].subclip(0, video_clips[1].duration - 1), fadedown(video_clips[1], video_clips[2]), video_clips[2].subclip(1.3, video_clips[2].duration - 1), video_clips[2].subclip(0, video_clips[2].duration - 1), fadein(video_clips[2], video_clips[3]), video_clips[3].subclip(1.3, video_clips[3].duration - 1), video_clips[3].subclip(0, video_clips[3].duration - 1), fun(video_clips[3], video_clips[4]), video_clips[4].subclip(1.3, video_clips[4].duration - 1), video_clips[4].subclip(0, vide

# not function

In [4]:
import cv2
import numpy as np
from moviepy.editor import *

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "motion.mp4"]

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    
    video_clips.append(video)

# Step 3: Define transition duration
transition_duration = 1  # Duration of the slide-in transition in seconds

# Step 4: Extract the last one second of clip1 and the first one second of clip2
clip1_last_second = video_clips[0].subclip(-1, None)
clip2_first_second = video_clips[1].subclip(0, 1.3)

# Check if aspect ratio of clip1_last_second is less than 1.78:1 and add black padding
if clip1_last_second.aspect_ratio < 1.78:
    width = int(1080 * clip1_last_second.aspect_ratio)
    padding = (1920 - width) // 2
    clip1_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip1_last_second.iter_frames()]
    # clip1_frames = [frame for frame in clip1_last_second.iter_frames()]
else:
    clip1_frames = [frame for frame in clip1_last_second.iter_frames()]

# Check if aspect ratio of clip2_first_second is less than 1.78:1 and add black padding
if clip2_first_second.aspect_ratio < 1.78:
    width = int(1080 * clip2_first_second.aspect_ratio)
    padding = (1920 - width) // 2
    clip2_frames = [cv2.copyMakeBorder(frame, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip2_first_second.iter_frames()]
    # clip2_frames = [frame for frame in clip2_first_second.iter_frames()]
else:
    
    clip2_frames = [frame for frame in clip2_first_second.iter_frames()]

# Step 6: Perform slide-in transition using numpy and cv2
transition_frames = []
height, width, _ = clip1_frames[0].shape

# Ensure the number of frames matches the expected duration
num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))

for i in range(num_frames):
    alpha = i / num_frames  # Linearly increasing alpha value

    # Calculate the translation distance based on the frame index
    tx_clip1 = -int((width / num_frames) * i)
    tx_clip2 = width - int((width / num_frames) * i)

    # Apply translation to clip1
    frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, tx_clip1], [0, 1, 0]]), (width, height))

    # Handle the case where clip2_frames has fewer frames than expected
    if i < len(clip2_frames):
        # Apply translation to clip2
        frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))
    else:
        # Use the last frame of clip2_frames for the remaining frames
        frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, tx_clip2], [0, 1, 0]]), (width, height))

    # Blend the frames to create the transition effect
    blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)

    transition_frames.append(blend_frame)

# Add remaining frames where clip2_first_second fills the entire screen
remaining_frames = clip2_frames[num_frames:]
transition_frames.extend(remaining_frames)

# Step 7: Concatenate the original clips and the transition frames to create the final video
transition_clip = ImageSequenceClip(transition_frames, fps=clip1_last_second.fps)
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), transition_clip.set_duration(transition_duration), video_clips[1].subclip(1.3, None)], method='compose')

# Step 8: Write the final video to a file
final_clip.write_videofile("lastslidein.mp4", codec="libx264", fps=final_clip.fps)



Moviepy - Building video lastslidein.mp4.
MoviePy - Writing audio in lastslideinTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video lastslidein.mp4



Moviepy - Done !
Moviepy - video ready lastslidein.mp4


# Down to UP

In [10]:
import cv2
import numpy as np
from moviepy.editor import *

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "motion.mp4"]

# Step 2: Create VideoFileClip objects from the videos
video_clips = []
for filename in video_filenames:
    video = VideoFileClip(filename)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    
    video_clips.append(video)

# Step 3: Define transition duration
transition_duration = 1  # Duration of the slide-down-to-up transition in seconds

# Step 4: Extract the last one second of clip1 and the first one second of clip2
clip1_last_second = video_clips[0].subclip(-1, None)
clip2_first_second = video_clips[1].subclip(0, 1)

# Check if aspect ratio of clip1_last_second is less than 1.78:1 and add black padding
if clip1_last_second.aspect_ratio < 1.78:
    width = int(1080 * clip1_last_second.aspect_ratio)
    padding = (1920 - width) // 2
    clip1_frames = [cv2.copyMakeBorder(frame, padding, padding, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip1_last_second.iter_frames()]
else:
    clip1_frames = [frame for frame in clip1_last_second.iter_frames()]

# Check if aspect ratio of clip2_first_second is less than 1.78:1 and add black padding
if clip2_first_second.aspect_ratio < 1.78:
    width = int(1080 * clip2_first_second.aspect_ratio)
    padding = (1920 - width) // 2
    clip2_frames = [cv2.copyMakeBorder(frame, padding, padding, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0]) for frame in clip2_first_second.iter_frames()]
else:
    clip2_frames = [frame for frame in clip2_first_second.iter_frames()]

# Step 6: Perform slide-down-to-up transition using numpy and cv2
transition_frames = []
height, width, _ = clip1_frames[0].shape

# Ensure the number of frames matches the expected duration
num_frames = min(len(clip1_frames), len(clip2_frames), int(transition_duration * clip1_last_second.fps))

for i in range(num_frames):
    alpha = i / num_frames  # Linearly increasing alpha value

    # Calculate the translation distance based on the frame index
    ty_clip1 = int((height / num_frames) * i)
    ty_clip2 = height - int((height / num_frames) * i)

    # Apply translation to clip1
    frame_clip1 = cv2.warpAffine(clip1_frames[i], np.float32([[1, 0, 0], [0, 1, ty_clip1]]), (width, height))

    # Handle the case where clip2_frames has fewer frames than expected
    if i < len(clip2_frames):
        # Apply translation to clip2
        frame_clip2 = cv2.warpAffine(clip2_frames[i], np.float32([[1, 0, 0], [0, 1, ty_clip2]]), (width, height))
    else:
        # Use the last frame of clip2_frames for the remaining frames
        frame_clip2 = cv2.warpAffine(clip2_frames[-1], np.float32([[1, 0, 0], [0, 1, ty_clip2]]), (width, height))

    # Blend the frames to create the transition effect
    blend_frame = cv2.addWeighted(frame_clip1, 1 - alpha, frame_clip2, alpha, 0)

    transition_frames.append(blend_frame)

# Add remaining frames where clip2_first_second fills the entire screen
remaining_frames = clip2_frames[num_frames:]
transition_frames.extend(remaining_frames)

# Step 7: Concatenate the original clips and the transition frames to create the final video
transition_clip = ImageSequenceClip(transition_frames, fps=clip1_last_second.fps)
final_clip = concatenate_videoclips([video_clips[0].subclip(0, video_clips[0].duration - 1), transition_clip.set_duration(transition_duration), video_clips[1].subclip(1, None)], method='compose')

# Step 8: Write the final video to a file
final_clip.write_videofile("slidedowntoup.mp4", codec="libx264", fps=final_clip.fps)


Moviepy - Building video slidedowntoup.mp4.
MoviePy - Writing audio in slidedowntoupTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video slidedowntoup.mp4



Moviepy - Done !
Moviepy - video ready slidedowntoup.mp4


In [42]:
from moviepy.editor import ImageClip, CompositeVideoClip
from moviepy.editor import transfx

EFFECT_DURATION = 0.3
CLIP_DURATION = 7

clip1 = ImageClip("m.jpeg").set_duration(CLIP_DURATION)
clip2 = ImageClip("n.jpeg").set_duration(CLIP_DURATION)
clip3 = ImageClip("nn.jpg").set_duration(CLIP_DURATION)

clips = [clip1, clip2, clip3]
num_clips = len(clips)

# Calculate the target width and height while maintaining aspect ratio
target_width = 1920
target_height = int(target_width * clip1.size[1] / clip1.size[0])

# Create a list to store the final clips
final_clips = []

for i, clip in enumerate(clips):
    start_time = (CLIP_DURATION - EFFECT_DURATION) * i

    # Apply slide out effect to the current clip
    slide_out_clip = clip.set_position("center").fx(transfx.slide_out, duration=EFFECT_DURATION, side="left")
    slide_out_clip = slide_out_clip.set_start(start_time)
    slide_out_clip = slide_out_clip.resize(width=target_width, height=target_height)

    final_clips.append(slide_out_clip)

    if i < num_clips - 1:
        # Apply slide in effect to the next clip
        slide_in_clip = clips[i+1].set_position("center").fx(transfx.slide_in, duration=EFFECT_DURATION, side="right")
        slide_in_clip = slide_in_clip.set_start(start_time + EFFECT_DURATION)
        slide_in_clip = slide_in_clip.resize(width=target_width, height=target_height)

        final_clips.append(slide_in_clip)

# Concatenate the final clips into a single video clip
final_video_clip = CompositeVideoClip(final_clips, size=(target_width, target_height))

# Write the video file
final_video_clip.write_videofile("3_final_clip.mp4", fps=10)



Moviepy - Building video 3_final_clip.mp4.
Moviepy - Writing video 3_final_clip.mp4



Moviepy - Done !
Moviepy - video ready 3_final_clip.mp4


In [14]:
import cv2

# Import the video
cap = cv2.VideoCapture("rose.mp4")

# Get the height and width of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Print the height and width of the video
print("Height:", height)
print("Width:", width)

Height: 720
Width: 406


In [57]:
import cv2

# Load the video
video_path = "fancy.mp4"
cap = cv2.VideoCapture(video_path)

# Get the width and height of the video
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("Height:", height)
print("Width:", width)
# Calculate the aspect ratio
aspect_ratio = width / height

print("Aspect Ratio: {:.2f}:{:.2f}".format(aspect_ratio, 1))

# Release the video capture
cap.release()


Height: 720.0
Width: 1280.0
Aspect Ratio: 1.78:1.00


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx

# Load the video clips
clip1 = VideoFileClip('fancy.mp4').subclip(0, 6).fx(vfx.fadein, 1).fx(vfx.fadeout, 1)
clip2 = VideoFileClip('motion.mp4').subclip(0, 6).fx(vfx.fadein, 1).fx(vfx.fadeout, 1)

# Resize the clips to match the aspect ratio of the first clip (fancy.mp4)
clip2 = clip2.resize(height=clip1.size[1])

# Concatenate the clips
combined = concatenate_videoclips([clip1, clip2])

# Write the final video file
combined.write_videofile('new.mp4')


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx

# Set the desired aspect ratio
desired_aspect_ratio = 16 / 9  # For example, 16:9

# Load the video clips
clip1 = VideoFileClip('fancy.mp4').subclip(0, 6).fx(vfx.fadein, 1).fx(vfx.fadeout, 1)
clip2 = VideoFileClip('1686498673.mp4').subclip(0, 6).fx(vfx.fadein, 1).fx(vfx.fadeout, 1)

# Calculate the current aspect ratios of the clips
aspect_ratio1 = clip1.size[0] / clip1.size[1]
aspect_ratio2 = clip2.size[0] / clip2.size[1]

# Resize the clips to match the desired aspect ratio
clip1_resized = clip1.resize(height=int(clip1.size[0] / desired_aspect_ratio))
clip2_resized = clip2.resize(height=int(clip2.size[0] / desired_aspect_ratio))

# Concatenate the resized clips
final_clip = concatenate_videoclips([clip1_resized, clip2_resized])

# Write the final video file
final_clip.write_videofile('new.mp4')

In [ ]:
import cv2
import numpy as np
def export_black_video(filename, width, height, duration):
  """Exports a full black video to the specified filename.

  Args:
    filename: The filename of the video to export.
    width: The width of the video.
    height: The height of the video.
    duration: The duration of the video in seconds.
  """

  video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
  black_frame = np.zeros((height, width, 3), dtype=np.uint8)
  for i in range(duration * 30):
    video.write(black_frame)
  video.release()

if __name__ == '__main__':
  export_black_video('black.mp4', 1280, 720, 20)


In [ ]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.resize import resize

# Load the video clip
clip = VideoFileClip("motion.mp4").subclip(0, 6)

# Calculate the desired height based on the 16:9 aspect ratio
desired_height = int(clip.w * 9 / 16)

# Resize the clip while maintaining the aspect ratio
resized_clip = resize(clip, height=desired_height)

# Write the resized clip to a new file
resized_clip.write_videofile("motion_resized.mp4")


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.video.fx.resize import resize

# Load the video clips
clip_fancy = VideoFileClip("fancy.mp4").subclip(0, 6)
# Load the video clip
clip = VideoFileClip("motion.mp4").subclip(0, 6)

# Calculate the desired height based on the 16:9 aspect ratio
desired_height = int(clip.w * 9 / 16)

# Resize the clip while maintaining the aspect ratio
resized_clip = resize(clip, height=desired_height)
# Concatenate the clips
combined_clip = concatenate_videoclips([clip_fancy, resized_clip])

# Write the combined clip to a new file
combined_clip.write_videofile("combined.mp4")


In [ ]:
from moviepy.editor import *

clip = VideoFileClip("motion.mp4").subclip(0,6)

combine = clips_array([[clip]])
cambiado = combine.fx(vfx.resize,(1920,1080),width= 1980)
cambiado.write_videofile("combina5v.mp4")

# PERFECT

In [43]:
from moviepy.editor import VideoFileClip, ColorClip, CompositeVideoClip

# Load the motion clip
clip_motion = VideoFileClip(r"fancy.mp4").subclip(0, 6)

# Define the width and height of the black background
background_width = 1080
background_height = 1920

# Create a black color clip as the background
background_clip = ColorClip((background_width, background_height), color=(0, 0, 0)).set_duration(6)

# Calculate the position to center the motion clip on the background
x_pos = (background_width - clip_motion.w) // 2
y_pos = (background_height - clip_motion.h) // 2
print(x_pos)
print(y_pos)
# Composite the motion clip onto the black background
final_clip = CompositeVideoClip([background_clip.set_position((x_pos, y_pos)), clip_motion.set_position((x_pos, y_pos))])

# Write the final clip to a new file
final_clip.write_videofile("motion_on_black_background.mp4", fps=clip_motion.fps)


-100
600
Moviepy - Building video motion_on_black_background.mp4.
MoviePy - Writing audio in motion_on_black_backgroundTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video motion_on_black_background.mp4



Moviepy - Done !
Moviepy - video ready motion_on_black_background.mp4


In [7]:
from moviepy.editor import VideoFileClip, ColorClip, CompositeVideoClip

# Load the motion clip
clip_motion = VideoFileClip(r"fancy.mp4").subclip(0, 6)

# Define the width and height of the black background
background_width = 1080
background_height = 1920

# Create a black color clip as the background
background_clip = ColorClip((background_width, background_height), color=(0, 0, 0)).set_duration(6)

# Calculate the position to center the motion clip on the background
x_pos = (background_width - clip_motion.w) // 2
y_pos = (background_height - clip_motion.h) // 2
print(x_pos)
print(y_pos)
# Composite the motion clip onto the black background
final_clip = CompositeVideoClip([background_clip.set_position((x_pos, y_pos)), clip_motion.set_position((x_pos, y_pos))])
# Set the target resolution to HD (1920x1080)
target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
combine = final_clip.resize(height=target_resolution[1], width=target_resolution[0])

# Write the final clip to a new file
combine.write_videofile("last.mp4", fps=20)

-100
600
Moviepy - Building video last.mp4.
MoviePy - Writing audio in lastTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video last.mp4



Moviepy - Done !
Moviepy - video ready last.mp4


In [35]:
from moviepy.editor import VideoFileClip, ColorClip, CompositeVideoClip


# Define the width and height of the black background
background_width = 1080
background_height = 1980

# Create a black color clip as the background

background_clip = ColorClip((background_width, background_height), color=(0, 0, 0)).set_duration(10)

# Calculate the position to center the motion clip on the background
x_pos = 0
y_pos = 1
clip_motion = VideoFileClip(r"motion.mp4").subclip(0, 6)
# Composite the motion clip onto the black background
final_clip = CompositeVideoClip([background_clip.set_position((x_pos, y_pos)), clip_motion.set_position((x_pos, y_pos))])

# Write the final clip to a new file
final_clip.write_videofile("my_reels.mp4", fps=20)

Moviepy - Building video my_reels.mp4.
MoviePy - Writing audio in my_reelsTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video my_reels.mp4



Moviepy - Done !
Moviepy - video ready my_reels.mp4


In [40]:
from moviepy.editor import VideoFileClip, ColorClip, CompositeVideoClip


# Define the width and height of the black background
background_width = 1080
background_height = 1980

# Create a black color clip as the background

background_clip = ColorClip((background_width, background_height), color=(0, 0, 0)).set_duration(10)

# Calculate the position to center the motion clip on the background
x_pos = 0
y_pos = 1
clip_motion = VideoFileClip(r"fancy.mp4").subclip(0, 6)
# Composite the motion clip onto the black background
final_clip = CompositeVideoClip([background_clip.set_position((x_pos, y_pos)), clip_motion.set_position((x_pos, y_pos))])

# Write the final clip to a new file
final_clip.write_videofile("my_reels.mp4", fps=20)

Moviepy - Building video my_reels.mp4.
MoviePy - Writing audio in my_reelsTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video my_reels.mp4



Moviepy - Done !
Moviepy - video ready my_reels.mp4


# Multiple screen

# Don't use 4k, it's so slow

In [40]:
from moviepy.editor import *

clip1 = VideoFileClip(r"D:\video\Video\Video\twice\twice.mp4").subclip(87, 93).margin(8)
clip2 = VideoFileClip("fancy.mp4").subclip(0, 6).margin(8)
clip3 = VideoFileClip(r"D:\video\Video\Video\twice\twice.mp4").subclip(87, 93).margin(8)
clip4 = VideoFileClip("fancy.mp4").subclip(0, 6).margin(8)
clip2.resize((1920,1080))
clip4.resize((1920,1080))
combine = clips_array([[clip1, clip2],
                       [clip3, clip4]])

# Set the target resolution to HD (1920x1080)
target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
combine = combine.resize(height=target_resolution[1], width=target_resolution[0])

# Write the combined clips to a new file
combine.write_videofile('combined1.mp4')


Moviepy - Building video combined1.mp4.
MoviePy - Writing audio in combined1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video combined1.mp4



Moviepy - Done !
Moviepy - video ready combined1.mp4


# Perfect export of different resolution

In [3]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx
from moviepy.editor import *
# clip1 = VideoFileClip(r"D:\video\Video\Video\twice\twice.mp").subclip(87, 93)
clip1 = VideoFileClip("feel.mp4",target_resolution=(1080, 1920)).subclip(0,6)
clip2 = VideoFileClip("dubu.mp4").subclip(0,6)

combine = concatenate_videoclips([clip1,clip2],method="compose")
# Set the target resolution to HD (1920x1080)
target_resolution = (1920, 1080)

# Resize the combined clip to the target resolution
combined = combine.resize(height=target_resolution[1], width=target_resolution[0])
combined.write_videofile('dahyun0.mp4')

Moviepy - Building video dahyun0.mp4.
MoviePy - Writing audio in dahyun0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video dahyun0.mp4



Moviepy - Done !
Moviepy - video ready dahyun0.mp4


# traget resolution

In [10]:
import cv2

# Load the video
video_path = "combined.mp4"
cap = cv2.VideoCapture(video_path)

# Define the target resolution
target_width = 1080
target_height = 1920

# Get the original video's width and height
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# Calculate the aspect ratio of the original video
aspect_ratio = width / height

# Calculate the new dimensions for resizing
if aspect_ratio > target_width / target_height:
    new_width = target_width
    new_height = int(target_width / aspect_ratio)
else:
    new_height = target_height
    new_width = int(target_height * aspect_ratio)

# Create an output video writer
output_path = "output_combined_hd.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to the target dimensions
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Write the resized frame to the output video
    out.write(resized_frame)

    # Display the resized frame
    cv2.imshow("Resized Frame", resized_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and output video writer
cap.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


In [62]:
import cv2

# Load the video
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        # End of the video
        break

    # Display the frame
    cv2.imshow("Reels Video", frame)

    # Check for the 'q' key to quit
    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

# Release the video capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [21]:
import threading

# Get the number of active threads
num_threads = threading.active_count()

print("Number of active threads:", num_threads)

Number of active threads: 7
